In [1]:
## Import Custom Functions and 
import functions_combined_BEST as ji
from functions_combined_BEST import ihelp, ihelp_menu,\
reload, inspect_variables

## IMPORT BS_DS PACKAGES
import bs_ds as bs
from bs_ds.imports import *
from pprint import pprint


# Import plotly and cufflinks for iplots
import plotly
import cufflinks as cf
cf.go_offline()

# Suppress warnings
import warnings
# warnings.filterwarnings('ignore')

# Set pd.set_options for tweet visibility
# pd.set_option('display.max_colwidth',100)

# Reload modules in case files have been updated
reload(ji)

bs_ds  v0.8.9 loaded.  Read the docs: https://bs-ds.readthedocs.io/en/latest/index.html
For convenient loading of standard modules use: from bs_ds.imports import *



Package,Handle,Description
bs_ds,bs,Custom data science bootcamp student package
matplotlib,mpl,Matplotlib's base OOP module with formatting artists
matplotlib.pyplot,plt,Matplotlib's matlab-like plotting module
numpy,np,scientific computing with Python
pandas,pd,High performance data structures and tools
seaborn,sns,High-level data visualization library based on matplotlib


In [2]:
filename = 'assets/data/twitter_df_for_searches.csv'
twitter_df = pd.read_csv(filename, index_col=0, parse_dates=True)


# TWITTER SEARCH

## Testing Used Functions


In [ ]:
reload(ji)
from_column = 'content_min_clean'
n=10
word='russia'
ascending=False
display_cols=['delta_price_class','delta_price','retweet_count','favorite_count','compound_score','source']
idx= ji.search_for_tweets_with_word(twitter_df=twitter_df,word=word,
                               from_column=from_column,display_n=n,
                               ascending=ascending,display_cols=display_cols,
                               return_index=True,display_df=False)

iloc_found = idx.reset_index(drop=True)


## Code for Main Functions

In [ ]:
reload(ji)
## move the control of the index to outside the call search function.
def interactive_tweet_search_redone(twitter_df,from_column = 'content_min_clean', n=10,
                             ascending=False,displayed_index=None, display_cols=['delta_price_class','delta_price','retweet_count',
                                                            'favorite_count','compound_score','source']):
    from ipywidgets import interactive,interact, interactive_output
    import ipywidgets as widgets
    from IPython.display import display
    
    
    search_field = widgets.Text(value='russia', placeholder='Enter word to search find tweets',
                                description='String:', disabled=False)
    n_tweets = widgets.IntText(value=n,description='# to display at once',disabled=False)
    search_bar = widgets.HBox(children=[search_field,n_tweets])
    button_next = widgets.Button(description='Next Page',
                                disabled=False)
    button_prev = widgets.Button(description='Prev Page',
                                disabled=True)
    buttons = widgets.HBox(children=[button_prev, button_next])
    
    ui=widgets.HBox(children=[search_bar, buttons])
   
    def click_next(change):
        update_display(next_page=True)
    button_next.on_click(click_next)
    
    def click_prev(change):
        pass
    button_prev.on_click(click_prev)
    
    def new_search(change):
        new_word = change.new
        new_iloc = call_search_function(word=new_word)
    
    
    def call_search_function(twitter_df=twitter_df,word=word, #search_field.value,
                             from_column=from_column,n=n_tweets.value, ascending=ascending,
                             display_cols=display_cols,return_index=False):#displayed_index=idx_returned,
        
        import functions_combined_BEST as ji
        idx_found = ji.search_for_tweets_with_word(twitter_df=twitter_df,word=word,
                                       from_column=from_column,display_n=n,
                                       ascending=ascending,display_cols=display_cols,
                                       return_index=True,display_df=False)
        
        iloc_index = idx_found.reset_index(drop=True)
        iloc_index.name='iloc'
        
        update_display(twitter_df=twitter_df, iloc_index=iloc_index)
        if return_index:
            return iloc_index
#         return idx_found
        
    def update_display(twitter_df=twitter_df, iloc_index=iloc_index,
                       display_n=n_tweets.value,start_n=None, end_n=None):
        
        
        ## PAGINATION
        if start_n  is None:
            cur = 0
        else:
            cur=start_n
        cur_end = cur+display_n+1    
        
        ## VARS TO DISPLAY
        total_results = len(iloc_index)  
        # Determine # of pages and indices to show for each
        pages_total = total_results//n
        if (total_results % n)>0:
            pages_total+=1
        capt_text =f"{total_results} found for {word}. Displaying results {cur} - {cur+n}" #Page {} of {pages_total}
                    
        
        # select indices
        dates_to_show = iloc_index[cur:cur_end]
        df_to_show = twitter_df.loc[dates_to_show]
        df_to_show.sort_index(ascending=False,inplace=True)

        style_and_display(df_to_show, capt_text=capt_text)
    
    def style_and_display(df_to_show, capt_text):

        ## Set table_style for display df.styler
        table_style =[{'selector':'caption',
                    'props':[('font-size','1.3em'),
                             ('color','darkblue'),
                             ('font-weight','semibold'),
                            ('text-align','left')]},
                     {'selector':'th',
                      'props':[('font-size','1.1em'),('text-align','center')]}]
        dfs_props = {'width':'400px', 'text-align':'center','padding':'2em','font-size':'1.2em'}

        ## Create dfs to display
        dfs = df_to_show.style.hide_index().\
        set_caption(capt_text).set_properties(
            subset=[from_column],**dfs_props).set_table_styles(table_style)

        display(dfs)#dfs)
#             return displayed_index       
        
        

    output = widgets.interactive_output(call_search_function,{'word':search_field,
                                                             'n':n_tweets})
    display(ui,output)
#     pass

interactive_tweet_search_redone(twitter_df)    

## Older, scrapped version

In [ ]:
reload(ji)

def interactive_tweet_search(twitter_df,from_column = 'content_min_clean', n=10,
                             ascending=False,displayed_index=None, display_cols=['delta_price_class','delta_price','retweet_count',
                                                            'favorite_count','compound_score','source']):
    from ipywidgets import interactive,interact, interactive_output
    import ipywidgets as widgets
    from IPython.display import display
    
    search_field = widgets.Text(value='russia', placeholder='Enter word to search find tweets',
                                description='String:', disabled=False)
    
    n_tweets = widgets.IntText(value=n,description='# to display at once',disabled=False)
    
    search_bar = widgets.HBox(children=[search_field,n_tweets])
    
    button_next = widgets.Button(description='Next Page',
                                disabled=False)
    
    button_prev = widgets.Button(description='Prev Page',
                                disabled=True)
    buttons = widgets.HBox(children=[button_prev, button_next])
    
    
    ## results style
    if displayed_index is None:
        idx_returned = None
    else:
        idx_returned=displayed_index

    ui=widgets.HBox(children=[search_bar, buttons])
    def call_search_function(twitter_df=twitter_df,word=search_field.value,
                             from_column=from_column,n=n_tweets.value, ascending=ascending,
                             display_cols=display_cols, display_df=True,
                            displayed_index=idx_returned,next_page=False,prev_page=False):
        
        import functions_combined_BEST as ji
        idx_found = ji.search_for_tweets_with_word(twitter_df=twitter_df,word=word,
                                       from_column=from_column,display_n=n,
                                       ascending=ascending,display_cols=display_cols,
                                       return_index=True,display_df=False)
        
        iloc_index = idx_found.reset_index(drop=True)
        iloc_index.name='iloc'
        
        ## PAGINATION?
        if displayed_index is not None and next_page:
            last_displayed = displayed_index.iloc[-1]

            cur = last_displayed+1
        else:
            cur = 0
        
        total_results = len(iloc_index)
        if cur < total_results: #and cur < nun:
            # url_params['offset'] = cur
            idx_show = iloc_index.iloc[cur:cur+n]         
            
#             iloc
            df_to_show = twitter_df.loc[idx_show]
            df_to_show.sort_index(ascending=False,inplace=True)
        

        
            # Determine # of pages and indices to show for each
            pages_total = total_results//n
            if (total_results % n)>0:
                pages_total+=1
                
            capt_text =f"{total_results} found for {word}. Displaying results {cur} - {cur+n}" #Page {} of {pages_total}
            
            ## Set table_style for display df.styler
            table_style =[{'selector':'caption',
                        'props':[('font-size','1.3em'),
                                 ('color','darkblue'),
                                 ('font-weight','semibold'),
                                ('text-align','left')]},
                         {'selector':'th',
                          'props':[('font-size','1.1em'),('text-align','center')]}]
        #                  {'selector':'td','props':[('pad','0.1em')]}]
            dfs_props = {'width':'400px', 'text-align':'center','padding':'2em','font-size':'1.2em'}
            dfs = df_to_show.style.hide_index().\
            set_caption(capt_text).set_properties(subset=[from_column],**dfs_props).set_table_styles(table_style)
    
            display(dfs)#dfs)
            return displayed_index       
        
        
    def click_next(change):
        up(displayed_index=displayed_index,next_page=True)
    button_next.on_click(click_next)
    
    def click_prev(change):
        pass
    output = widgets.interactive_output(call_search_function,{'word':search_field,
                                                             'n':n_tweets})
    display(ui,output)
#     pass

interactive_tweet_search(twitter_df)    